In [1]:
# Import libraries
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
# # Read News and DJIA Data
# df_news = pd.read_csv('/kaggle/input/financialnews/Combined_News_DJIA.csv')
# df_stock = pd.read_csv('/kaggle/input/financialnews/upload_DJIA_table.csv')
df_news = pd.read_csv('/kaggle/input/masterthesis-00-datagathering/SP500_all.csv')

In [3]:
# check data
display(df_news.info())
# print('______________________________________________')
# display(df_stock.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13380 entries, 0 to 13379
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  13380 non-null  int64  
 1   date        13380 non-null  object 
 2   Open        13380 non-null  float64
 3   Close       13380 non-null  float64
 4   High        13380 non-null  float64
 5   Low         13380 non-null  float64
 6   Volume      13380 non-null  int64  
 7   release     13378 non-null  object 
 8   headline    13378 non-null  object 
 9   text        13378 non-null  object 
dtypes: float64(4), int64(2), object(4)
memory usage: 1.0+ MB


None

In [4]:
# df_news.drop('Unnamed: 0', axis=1, inplace=True)
# df_news.drop('Unnamed: 0', axis=1, inplace=True)
# df_news.drop('text', axis=1, inplace=True)

df_news = df_news[['date','headline']]

In [5]:
# Adjust display settings

pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

# Get 5 random samples with reproducibility
display(df_news.sample(n=5, random_state=42))

,date,headline
9282,2021-04-08,Nasdaq Futures Up 125 Pts; Jobless Claims Set to Resume Decline Later
6991,2020-02-26,Stocks - U.S. Futures Waver as Wall St Attempts Bounce After Epic Rout
5643,2019-04-15,Wall Street slips as bank earnings disappoint
624,2016-08-16,Wall St. slips from highs after Fed officials' rate comments
10735,2021-12-13,Apple closes in on $3 trillion market value


In [6]:
df_news.fillna('', inplace=True)

In [7]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
import re
import html

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Define the stop words
stop_words = set(stopwords.words('english'))

# Define a function to preprocess the headlines
def preprocess_text(text, mode="simple"):
    # Convert the text to lower case
    #text = text.lower()
    
    # Remove html entities
    text = html.unescape(text)
    
    # Remove byte-string prefix and content
    text = re.sub(r"b'", "", text)
    text = re.sub(r'b"', "", text)
    text = re.sub(r"\r", "", text)
    text = re.sub(r"\n", "", text)
    text = re.sub(r"\\", "", text)
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    
    # Remove numeric values
    text = re.sub(r'\d+', '', text)
    
    if mode == "advanced":   
    
        # Remove punctuation
        text = re.sub(r'[^\w\s]', '', text)

        # Tokenize the text
        text = text.split()

        # Remove stop words and lemmatize the words
        text = [lemmatizer.lemmatize(word) for word in text if word not in stop_words]
        
        # Join the words back into a single string
        text = ' '.join(text)
        
    else:
        
        # Join the words back into a single string
        text = ''.join(text)
    
    return text

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [8]:
# # New columns for preprocessed headlines
# new_cols_simple = [f'Top{i}_Preprocessed' for i in range(1, 26)]
# # new_cols_advanced = [f'Top{i}_Preprocessed_Advanced' for i in range(1, 26)]

# # Apply the function to each headline
# for i in range(1, 26):
#     col_name = f'Top{i}'
#     new_col_simple = f'Top{i}_Preprocessed'
# #     new_col_advanced = f'Top{i}_Preprocessed_Advanced'
    
#     df_news[new_col_simple] = df_news[col_name].apply(preprocess_text, mode="simple")
# #     df_news[new_col_advanced] = df_news[col_name].apply(preprocess_text, mode="advanced")

df_news['headline_preprocessed'] = df_news['headline'].apply(preprocess_text, mode="simple")

In [9]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13380 entries, 0 to 13379
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   date                   13380 non-null  object
 1   headline               13380 non-null  object
 2   headline_preprocessed  13380 non-null  object
dtypes: object(3)
memory usage: 313.7+ KB


In [10]:
# from transformers import BertTokenizer, BertForSequenceClassification
# from transformers import pipeline

# finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
# tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

# nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer, top_k = 1)

# sentences = df_news['headline_preprocessed'][:10].tolist()
# results = nlp(sentences)
# print(results)  #LABEL_0: neutral; LABEL_1: positive; LABEL_2: negative

# sentences = df_news['headline'][:10].tolist()
# results = nlp(sentences)
# print(results)  #LABEL_0: neutral; LABEL_1: positive; LABEL_2: negative

In [11]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax

# Load pre-trained model and tokenizer from Hugging Face
model_name = 'yiyanghkust/finbert-tone'  
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def predict_sentiment_finbert(data, batch_size=32):
    """
    Predicts the sentiment of text data in a DataFrame using FinBERT, with batching.
    
    Args:
        data (pd.Series): Series containing the text data.
        batch_size (int): The number of samples to process in each batch.
        
    Returns:
        pd.DataFrame: A DataFrame with sentiment scores and labels.
    """
    scores = []
    labels = []
    prob_score = []

    # Tokenize text and convert to input format for BERT
    tokenized_output = tokenizer(data.tolist(), 
                                 padding=True, 
                                 truncation=True, 
                                 return_tensors="pt", 
                                 max_length=512)
    input_ids = tokenized_output['input_ids']
    attention_mask = tokenized_output['attention_mask']
    
    # Move to device
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)

    # Process text in batches
    for i in range(0, len(input_ids), batch_size):
        inputs = input_ids[i : i + batch_size]

        # Model inference
        with torch.no_grad():
            outputs = model(inputs, attention_mask=attention_mask[i : i + batch_size])
        
        # Get probabilities
        probs = softmax(outputs.logits, dim=1).cpu().numpy()
        
        # Convert to a score between -1 and 1
        sentiment_scores = probs[:, 1] - probs[:, 2]  # LABEL_0: neutral; LABEL_1: positive; LABEL_2: negative
        
        # Get the sentiment label (assuming LABEL_0: neutral; LABEL_1: positive; LABEL_2: negative)
        sentiment_labels = ['Neutral', 'Positive', 'Negative']
        sentiment_label_indices = probs.argmax(axis=1)
        batch_labels = [sentiment_labels[idx] for idx in sentiment_label_indices]
        
        scores.extend(sentiment_scores)
        labels.extend(batch_labels)
        prob_score.extend(probs)

    results = pd.DataFrame({
        'headline': data,
        'finbert_score': scores, 
        'label': labels,
        'probs': prob_score
    }, index=data.index)

    return results

df_results = predict_sentiment_finbert(df_news['headline'])

In [12]:
df_news = pd.concat([df_news, df_results['finbert_score']], axis=1)

In [13]:
df_news.head(10)

,date,headline,headline_preprocessed,finbert_score
0,2016-01-04,Wall Street begins year sharply lower after China selloff,Wall Street begins year sharply lower after China selloff,-0.999072
1,2016-01-05,Asian shares mixed with China and Australia services PMIs weak,Asian shares mixed with China and Australia services PMIs weak,-0.999880
2,2016-01-05,Wall Street lower as investors turn cautious; Apple drags,Wall Street lower as investors turn cautious; Apple drags,-0.999807
3,2016-01-05,Investing.com sentiment: speculators less bearish on S&P 500,Investing.com sentiment: speculators less bearish on S&P,-0.999070
4,2016-01-05,Global share market settles after stormy start to year,Global share market settles after stormy start to year,-0.985360
5,2016-01-05,Shares in Asia mixed with Shanghai recovering after morning drop,Shares in Asia mixed with Shanghai recovering after morning drop,-0.262252
6,2016-01-06,Asian shares down sharply as trading halt again hits Shanghai,Asian shares down sharply as trading halt again hits Shanghai,-0.999978
7,2016-01-07,Shares in Asia mixed with Shanghai recovering after morning plunge,Shares in Asia mixed with Shanghai recovering after morning plunge,-0.659930
8,2016-01-07,"Dow, S&P off to worst four-day Jan start ever as China fears grow","Dow, S&P off to worst four-day Jan start ever as China fears grow",-0.993809
9,2016-01-07,U.S. stocks plunge 2% as continuing China woes spill into global markets,U.S. stocks plunge % as continuing China woes spill into global markets,-0.999999


In [14]:
# import torch
# import pandas as pd
# from transformers import BertTokenizer, BertForSequenceClassification
# from torch.nn.functional import softmax

# # Load pre-trained model and tokenizer from Hugging Face
# model_name = 'yiyanghkust/finbert-tone'  # Example FinBERT model, adjust as needed
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertForSequenceClassification.from_pretrained(model_name)

# # Use GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# def predict_sentiment_finbert(data, batch_size=32):
#     """
#     Predicts the sentiment of text data in a DataFrame using FinBERT, with batching.
    
#     Args:
#         data (pd.Series): Series containing the text data.
#         batch_size (int): The number of samples to process in each batch.
        
#     Returns:
#         pd.Series: A series of sentiment scores between -1 and 1.
#     """
#     scores = []

#     # Tokenize text and convert to input format for BERT
#     input_ids = tokenizer(data.tolist(), 
#                           padding=True, 
#                           truncation=True, 
#                           return_tensors="pt", 
#                           max_length=512)['input_ids']
    
#     # Move to device
#     input_ids = input_ids.to(device)

#     # Process text in batches
#     for i in range(0, len(input_ids), batch_size):
#         inputs = input_ids[i : i + batch_size]

#         # Model inference
#         with torch.no_grad():
#             outputs = model(inputs)
        
#         # Get probabilities
#         probs = softmax(outputs.logits, dim=1).cpu().numpy()

#         # Convert to a score between -1 and 1
#         sentiment_scores = probs[:, 2] - probs[:, 0]  # Assuming 0: negative, 1: neutral, 2: positive
        
#         scores.extend(sentiment_scores)

#     return pd.Series(scores, index=data.index)


# # Call the function and add the resulting scores to df_news
# df_news['finbert_score'] = predict_sentiment_finbert(df_news['headline'])

In [15]:
pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 5.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def predict_sentiment_vader(dataframe, text_column):
    """
    Predicts the sentiment of text data in a DataFrame using VADER.
    
    Args:
        dataframe (pd.DataFrame): DataFrame containing the text data.
        text_column (str): The name of the column containing the text data.
        
    Returns:
        pd.Series: A series of sentiment scores between -1 and 1.
    """
    analyser = SentimentIntensityAnalyzer()
    
    # Compute sentiment scores
    scores = dataframe[text_column].apply(lambda x: analyser.polarity_scores(x)['compound'])
    
    return scores


# Call the function and add the resulting scores to df_news
df_news['vader_score'] = predict_sentiment_vader(df_news, 'headline_preprocessed')


In [17]:
df_news

,date,headline,headline_preprocessed,finbert_score,vader_score
0,2016-01-04,Wall Street begins year sharply lower after China selloff,Wall Street begins year sharply lower after China selloff,-0.999072,-0.2960
1,2016-01-05,Asian shares mixed with China and Australia services PMIs weak,Asian shares mixed with China and Australia services PMIs weak,-0.999880,-0.1779
2,2016-01-05,Wall Street lower as investors turn cautious; Apple drags,Wall Street lower as investors turn cautious; Apple drags,-0.999807,-0.5106
3,2016-01-05,Investing.com sentiment: speculators less bearish on S&P 500,Investing.com sentiment: speculators less bearish on S&P,-0.999070,0.0000
4,2016-01-05,Global share market settles after stormy start to year,Global share market settles after stormy start to year,-0.985360,0.2960
...,...,...,...,...,...
13375,2022-12-28,Tesla shares extend losses on demand worries in China,Tesla shares extend losses on demand worries in China,-1.000000,-0.4767
13376,2022-12-29,Stock market today: Dow ends higher as dip-buying fever returns to bruised tech,Stock market today: Dow ends higher as dip-buying fever returns to bruised tech,0.999984,0.0000
13377,2022-12-29,S&P 500 jumps as dip-buyers piled into tech,S&P jumps as dip-buyers piled into tech,0.390672,0.0000
13378,2022-12-29,Stock markets extend gains as an end-of-year relief rally kicks off,Stock markets extend gains as an end-of-year relief rally kicks off,0.999997,0.7351


In [18]:
df_news.to_csv("sentiment.csv")

In [19]:
df_stock = pd.read_csv('/kaggle/input/masterthesis-00-datagathering/SP500_stock.csv')
df_stock.drop('Unnamed: 0', axis=1, inplace=True)

# Calculate the average sentiment per day
df_news = df_news.groupby('date')[['finbert_score', 'vader_score']].mean()
df = df_news.merge(df_stock, on= "date")

df.to_csv("stock_sentiment.csv")